In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score



# Regression Models Lab
## Linear and logistic regression: theory and practice

In this lab you'll revisit and expand on your knowledge of modelling in general, as well as the fundamentals of linear and logistic regression. As a reminder, _linear regression_ is a regression model (regressor), and _logistic regression_ is a classification model (classifier).

This time, you'll use generated data, in order to separate some of the complexity of handling various datasets from inspecting and evaluating models.

**Use vectorization as much as possible!** You should be able to complete the lab using for-loops only to track the training steps.

### Problem 1. Generate some data for multiple linear regression (1 point)
As an expansion to the lecture, you'll create a dataset and a model.

Create a dataset of some (e.g., 50-500) observations of several (e.g., 5-20) independent features. You can use random generators for them; think about what distributions you'd like to use. Let's call them $x_1, x_2, ..., x_m$. The data matrix $X$ you should get should be of size $n \times m$. It's best if all features have different ranges.

Create the dependent variable by assigning coefficients $\bar{a_1}, \bar{a_2}, ..., \bar{a_m}, \bar{b}$ and calculating $y$ as a linear combination of the input features. Add some random noise to the functional values. I've used bars over coefficients to avoid confusion with the model parameters later.

Save the dataset ($X$ and $y$), and "forget" that the coefficients have ever existed. "All" you have is the file and the implicit assumption that there is a linear relationship between $X$ and $y$.

In [22]:
n_samples = 300
n_features = 10

X = np.column_stack([
    np.random.normal(3, 1, n_samples),
    np.random.uniform(0, 10, n_samples),
    np.random.exponential(1, n_samples),
    np.random.normal(50, 5, n_samples),
    np.random.uniform(-100, 100, n_samples),
    np.random.normal(10, 2, n_samples),
    np.random.beta(2, 5, n_samples),
    np.random.poisson(5, n_samples),
    np.random.gamma(2, 2, n_samples),
    np.random.binomial(10, 0.3, n_samples),
])

true_coefficients = np.random.uniform(-5, 5, n_features)
noise = np.random.normal(0, 1, n_samples)
y = X @ true_coefficients + noise

data = pd.DataFrame(X, columns=[f"x{i+1}" for i in range(X.shape[1])])
data["y"] = y

data

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,y
0,4.137545,3.825109,0.376041,41.445207,92.330760,11.315141,0.420754,7.0,6.256391,1.0,57.285716
1,3.777992,6.275629,1.095383,53.462824,-21.149480,9.913548,0.265124,4.0,4.726141,4.0,116.965343
2,2.952007,5.673085,3.663421,44.797434,-8.171268,9.864522,0.209517,3.0,6.907069,2.0,70.136096
3,3.224239,3.796216,0.100067,57.683033,75.307880,9.727270,0.294789,8.0,3.919697,1.0,117.715124
4,3.765803,3.370892,2.068343,42.433723,76.342059,9.269243,0.032575,2.0,3.491453,4.0,86.785033
...,...,...,...,...,...,...,...,...,...,...,...
295,3.444345,3.995054,0.127938,48.769992,42.863897,15.497256,0.084062,5.0,2.614264,3.0,107.125330
296,1.971352,4.027595,2.417250,43.721920,-73.330864,10.503673,0.136991,10.0,1.678107,4.0,85.939197
297,2.497212,7.496564,0.843873,57.742011,43.389594,10.630348,0.309603,6.0,9.349509,1.0,88.969308
298,2.495254,7.115487,0.237640,55.713926,15.408648,11.104123,0.268552,3.0,3.931404,2.0,125.426383


In [23]:
data.to_csv("dataset", index=False)

In [24]:
data = pd.read_csv('dataset')
data

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,y
0,4.137545,3.825109,0.376041,41.445207,92.330760,11.315141,0.420754,7.0,6.256391,1.0,57.285716
1,3.777992,6.275629,1.095383,53.462824,-21.149480,9.913548,0.265124,4.0,4.726141,4.0,116.965343
2,2.952007,5.673085,3.663421,44.797434,-8.171268,9.864522,0.209517,3.0,6.907069,2.0,70.136096
3,3.224239,3.796216,0.100067,57.683033,75.307880,9.727270,0.294789,8.0,3.919697,1.0,117.715124
4,3.765803,3.370892,2.068343,42.433723,76.342059,9.269243,0.032575,2.0,3.491453,4.0,86.785033
...,...,...,...,...,...,...,...,...,...,...,...
295,3.444345,3.995054,0.127938,48.769992,42.863897,15.497256,0.084062,5.0,2.614264,3.0,107.125330
296,1.971352,4.027595,2.417250,43.721920,-73.330864,10.503673,0.136991,10.0,1.678107,4.0,85.939197
297,2.497212,7.496564,0.843873,57.742011,43.389594,10.630348,0.309603,6.0,9.349509,1.0,88.969308
298,2.495254,7.115487,0.237640,55.713926,15.408648,11.104123,0.268552,3.0,3.931404,2.0,125.426383


### Working with Linear Regression model

In [25]:
data = pd.read_csv('dataset')
X = data.drop('y', axis=1)
y = data['y']

In [26]:
# Build and fit the model
model = LinearRegression()
model.fit(X, y)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [27]:
y_pred = model.predict(X)
mse = mean_squared_error(y, y_pred)
r2 = r2_score(y, y_pred)

print("Mean Squared Error:", mse)
print("R² Score:", r2)
print("Coef:", model.coef_)
print("Intercept:", model.intercept_)

Mean Squared Error: 1.0145347187734837
R² Score: 0.9980396976176283
Coef: [ 0.75288913 -2.33778049 -3.97988534  3.13512763 -0.09862652 -0.4712644
 -0.26264978 -3.15018841 -4.75034751  0.55267123]
Intercept: -0.22558157461330097


### Problem 2. Check your assumption (1 point)
Read the dataset you just saved (this is just to simulate starting a new project). It's a good idea to test and verify our assumptions. Find a way to check whether there really is a linear relationship between the features and output.

### Problem 3. Figure out the modelling function (1 point)
The modelling function for linear regression is of the form
$$ \tilde{y} = \sum_{i=1}^{m}a_i x_i + b $$

If you want to be clever, you can find a way to represent $b$ in the same way as the other coefficients.

Write a Python function which accepts coefficients and data, and ensure (test) it works correctly.

### Problem 4. Write the cost function and compute its gradients (1 point)
Use MSE as the cost function $J$. Find a way to compute, calculate, or derive its gradients w.r.t. the model parameters $a_1, ..., a_m, b$

Note that computing the cost function value and its gradients are two separate operations. Quick reminder: use vectorization to compute all gradients (maybe with the exception of $\frac{\partial J}{\partial b}$) at the same time.

### Problem 5. Perform gradient descent (1 point)
Perform weight updates iteratively. Find a useful criterion for stopping. For most cases, just using a fixed (large) number of steps is enough.

You'll need to set a starting point (think about which one should be good, and how it matters); and a learning rate.

### Problem 6. Do other cost functions work? (2 points)
Repeat the process in problems 4 and 5 with MAE, and then again - with the [Huber loss](https://en.wikipedia.org/wiki/Huber_loss). Both of them are less sensitive to outliers / anomalies than MSE); with the Huber loss function being specifically made for datasets with outliers.

Explain your findings. Is there a cost function that works much better? How about speed of training (measured in wall time)?

### Problem 7. Experiment with the learning rate (1 point)
Use your favorite cost function. Run several "experiments" with different learning rates. Try really small, and really large values. Observe and document your findings.

### Problem 8. Generate some data for classification (1 point)
You'll need to create two clusters of points (one cluster for each class). I recomment using `scikit-learn`'s `make_blobs()` ([info](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_blobs.html)). Use as many features as you used in problem 1.

### Problem 9. Perform logistic regression (1 point)
Reuse the code you wrote in problems 3-7 as much as possible. If you wrote vectorized functions with variable parameters - you should find this easy. If not - it's not too late to go back and refactor your code.

The modelling function for logistic regression is
$$ \tilde{y} = \frac{1}{1+\exp{(-\sum_{i=1}^{m}a_i x_i + b)}}$$. Find a way to represent it using as much of your previous code as you can.

The most commonly used loss function is the [cross-entropy](https://en.wikipedia.org/wiki/Cross-entropy).

Experiment with different learning rates, basically repeating what you did in problem 7.

### * Problem 10. Continue experimenting and delving deep into ML
You just saw how modelling works and how to implement some code. Some of the things you can think about (and I recommend you pause and ponder on some of them are):
* Code: OOP can be your friend sometimes. `scikit-learn`'s models have `fit()`, `predict()` and `score()` methods.
* Data: What approaches work on non-generated data?
* Evaluation: How well do different models (and their "settings" - hyperparameters) actually work in practice? How do we evaluate a model in a meaningful way?
* Optimization - maths: Look at what `optimizers` (or solvers) are used in `scikit-learn` and why. Many "tricks" revolve around making the algorithm converge (finish) in fewer iterations, or making it more numerically stable.
* Optimization - code: Are there ways to make the code run fastr?